# Data Cleaning and Reshaping

Clean and reshape the survey question mapping data:
1. Remove exact duplicate rows
2. Melt wide format to long format (question-survey pairs)
3. Remove duplicate question-survey pairs
4. Flag potential multi-level questions
5. Basic quality checks

In [12]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_colwidth', 100)

## Load Data

In [13]:
# Load the clean mapping file
df = pd.read_csv('../data/raw/PublicSurveyQuestionsMap.csv', dtype=str)

print(f"Initial shape: {df.shape}")
print(f"Questions: {len(df)}")

survey_cols = [col for col in df.columns if col != 'Question']
print(f"Survey columns: {len(survey_cols)}")

Initial shape: (6987, 49)
Questions: 6987
Survey columns: 48


## Check for Exact Duplicate Rows in Wide Format

In [14]:
# Check if there are any completely duplicate rows
duplicate_rows = df.duplicated()
num_duplicates = duplicate_rows.sum()

print(f"Number of exact duplicate rows: {num_duplicates}")

if num_duplicates > 0:
    print("\nExample duplicates:")
    print(df[duplicate_rows][['Question']].head())
    df = df.drop_duplicates()
    print(f"\nRemoved {num_duplicates} duplicate rows")
    print(f"New shape: {df.shape}")

Number of exact duplicate rows: 254

Example duplicates:
                                                                       Question
150                                                                         NaN
216                                                                         NaN
223                                                                         NaN
384                                                                         NaN
526  What is the highest grade or level of school this caregiver has completed?

Removed 254 duplicate rows
New shape: (6733, 49)


## Melt to Long Format

In [15]:
# Melt from wide to long format
# Creates one row per question-survey pair

df_long = df.melt(
    id_vars=['Question'],
    value_vars=survey_cols,
    var_name='survey',
    value_name='marker'
)

print(f"Shape after melting: {df_long.shape}")
print(f"\nFirst few rows:")
print(df_long.head(10))

Shape after melting: (323184, 3)

First few rows:
                                                                                              Question  \
0             At any time during this school year did you attend a public or private school in person?   
1  At any time during this school year did you receive any online schooling or virtual learning pro...   
2  At any time during this school year did you receive homeschooling instead of being enrolled in a...   
3    At any time during this school year did you attend a homeschool cooperative, or co-op, in person?   
4  Did you receive [online schooling or virtual learning/homeschooling instead of being enrolled in...   
5  For this next question, I’m going to read a list of 7 possible reasons that some families decide...   
6                                       How many different schools have you attended this school year?   
7  During the time you were homeschooled this school year, what grade would you have been enrolled ...

In [16]:
# Keep only rows where the marker is not null (question appears in that survey)
df_long = df_long[df_long['marker'].notna()].copy()

print(f"\nShape after filtering to actual question-survey pairs: {df_long.shape}")
print(f"Total question-survey pairs: {len(df_long):,}")


Shape after filtering to actual question-survey pairs: (6732, 3)
Total question-survey pairs: 6,732


In [17]:
# Drop the marker column - we don't need it anymore
df_long = df_long.drop(columns=['marker'])

print(f"\nColumns: {df_long.columns.tolist()}")
print(f"\nSample rows:")
print(df_long.head(20))


Columns: ['Question', 'survey']

Sample rows:
                                                                                               Question  \
0              At any time during this school year did you attend a public or private school in person?   
1   At any time during this school year did you receive any online schooling or virtual learning pro...   
2   At any time during this school year did you receive homeschooling instead of being enrolled in a...   
3     At any time during this school year did you attend a homeschool cooperative, or co-op, in person?   
4   Did you receive [online schooling or virtual learning/homeschooling instead of being enrolled in...   
5   For this next question, I’m going to read a list of 7 possible reasons that some families decide...   
6                                        How many different schools have you attended this school year?   
7   During the time you were homeschooled this school year, what grade would you have been enroll

## Check for Duplicate Question-Survey Pairs

In [18]:
# Check if any question appears multiple times in the same survey
duplicates = df_long.duplicated(subset=['Question', 'survey'])
num_duplicates = duplicates.sum()

print(f"Duplicate question-survey pairs: {num_duplicates}")

if num_duplicates > 0:
    print("\nExamples of duplicate pairs:")
    dup_pairs = df_long[duplicates]
    print(dup_pairs.head(10))
    
    # Remove duplicates
    df_long = df_long.drop_duplicates(subset=['Question', 'survey'])
    print(f"\nRemoved {num_duplicates} duplicate question-survey pairs")
    print(f"New shape: {df_long.shape}")

Duplicate question-survey pairs: 0


## Add Question IDs

In [19]:
# Create a mapping of unique questions to IDs
unique_questions = df_long['Question'].unique()
question_to_id = {q: i for i, q in enumerate(unique_questions, start=1)}

df_long['question_id'] = df_long['Question'].map(question_to_id)

print(f"Total unique questions: {len(unique_questions):,}")
print(f"Total question-survey pairs: {len(df_long):,}")

Total unique questions: 6,296
Total question-survey pairs: 6,732


## Flag Multi-Level Questions

In [20]:
# Identify questions that might be multi-level (a., b., c. patterns)
# These are questions that combine multiple sub-questions

def has_multilevel_pattern(text):
    """Check if question contains multi-level enumeration patterns"""
    if pd.isna(text):
        return False
    
    patterns = [
        r'\ba\.\s',  # a. 
        r'\bb\.\s',  # b. 
        r'\s[a-z]\)\s',  # a) b) c)
        r'\(a\)',  # (a) (b) (c)
        r'\s[A-Z]\.\s',  # A. B. C.
    ]
    
    for pattern in patterns:
        if re.search(pattern, text):
            return True
    return False

df_long['is_multilevel'] = df_long['Question'].apply(has_multilevel_pattern)

num_multilevel = df_long['is_multilevel'].sum()
pct_multilevel = (num_multilevel / len(df_long)) * 100

print(f"Multi-level question-survey pairs: {num_multilevel:,} ({pct_multilevel:.1f}%)")

Multi-level question-survey pairs: 123 (1.8%)


In [21]:
# Show examples of multi-level questions
if num_multilevel > 0:
    print("\nExamples of multi-level questions:")
    multilevel_samples = df_long[df_long['is_multilevel']].drop_duplicates(subset=['Question'])
    for idx, row in enumerate(multilevel_samples.head(5).itertuples(), 1):
        print(f"\n{idx}. [{row.survey}]")
        print(f"   {row.Question[:300]}...")


Examples of multi-level questions:

1. [Identity Theft Supplement (ITS)]
   45. Did you feel any of the following ways for A MONTH OR MORE because of misuse of your personal information? a. Worried or anxious? b. Angry? c. Sad or depressed? d. Vulnerable? e. Violated? f. Like you couldn’t trust people? g. Unsafe? h. Some other way?...

2. [Identity Theft Supplement (ITS)]
   47. Did you experience any of the following physical problems caused by misuse of your personal information for A MONTH OR MORE? Did you experience... a. Headaches? b. Trouble sleeping? c. Changes in your eating or drinking habits? d. Upset stomach? e. Fatigue? f. High blood pressure? g. Muscle tens...

3. [Identity Theft Supplement (ITS)]
   58. Other than anything we have already talked about, have you experienced any of the following problems as a result of misuse of your personal information? Have you… a. Had credit related problems, such as having to repeatedly correct the same information on your credit repo

## Question Length Analysis

In [22]:
# Add question length
df_long['question_length'] = df_long['Question'].str.len()

print("Question length statistics:")
print(df_long['question_length'].describe())

Question length statistics:
count    6731.000000
mean      134.110088
std       127.772618
min         8.000000
25%        66.000000
50%       102.000000
75%       155.000000
max      1808.000000
Name: question_length, dtype: float64


In [23]:
# Check for extremely short questions that might be fragments
very_short = df_long[df_long['question_length'] < 20].drop_duplicates(subset=['Question'])
print(f"\nUnique very short questions (<20 chars): {len(very_short)}")

if len(very_short) > 0:
    print("\nExamples:")
    for row in very_short.head(10).itertuples():
        print(f"  - '{row.Question}' [{row.survey}]")


Unique very short questions (<20 chars): 22

Examples:
  - 'What is your sex?' [National Survey of Children's Health Topical Questionaire (Children, 0-5 years)]
  - 'What is your age?' [National Survey of Children's Health Topical Questionaire (Children, 0-5 years)]
  - 'What is your race?' [National Teacher and Principal Survey (NTPS) School Questionnaire]
  - 'Enter age' [National Health Interview Survey (NHIS)]
  - 'How old is ^SCNAME?' [National Health Interview Survey (NHIS)]
  - '^SCNAME ^SOLITARY?' [National Health Interview Survey (NHIS)]
  - 'What work was done?' [Consumer Expenditure Survey (CE)]
  - 'What is the state?' [Consumer Expenditure Survey (CE)]
  - 'Did we miss anyone?' [Survey of Income and Program Participation (SIPP)]
  - 'What is ... race?' [Survey of Income and Program Participation (SIPP)]


In [24]:
# Check for extremely long questions
very_long = df_long[df_long['question_length'] > 500].drop_duplicates(subset=['Question'])
print(f"Unique very long questions (>500 chars): {len(very_long)}")

if len(very_long) > 0:
    print("\nExamples:")
    for row in very_long.head(3).itertuples():
        print(f"\n[{row.survey}]")
        print(f"{row.Question[:300]}...\n")

Unique very long questions (>500 chars): 116

Examples:

[School Crime Supplement (SCS)/National Crime Victimization Survey (NCVS)]
For this next question, I’m going to read a list of 7 possible reasons that some families decided to homeschool rather than enroll in public or private school. You can say yes to more than one reason but please choose ones that best fit the reason or reasons your family decided to homeschool. Becaus...


[School Crime Supplement (SCS)/National Crime Victimization Survey (NCVS)]
During this school year, have you participated in any of the following activities sponsored by your school: Spirit groups, for example, Cheerleading, Dance Team, or Pep Club? Athletic teams at school? Performing arts, for example Band, Choir, Orhcestra, or Drama? Academic clubs, for example, Debate ...


[School Crime Supplement (SCS)/National Crime Victimization Survey (NCVS)]
Now I have some questions about what students do during school that make you feel bad or are hurtful to yo

## Survey Distribution

In [25]:
# Questions per survey
survey_counts = df_long['survey'].value_counts().sort_values(ascending=False)

print("Top 15 surveys by question count:")
print(survey_counts.head(15))
print(f"\nTotal surveys: {len(survey_counts)}")

Top 15 surveys by question count:
survey
Survey of Income and Program Participation (SIPP)                                    1158
Consumer Expenditure Survey (CE)                                                     1058
National Health Interview Survey (NHIS)                                               764
American Housing Survey (AHS)                                                         736
Food Acquisition and Purchase Survey (FoodAPS) Initial Interview/Household Survey     242
Current Population Survey (CPS)                                                       185
National Sample Survey of Registered Nurses (NSSRN)                                   180
National Survey of Children's Health Topical Questionaire (Children, 0-5 years)       160
National Survey of Children's Health Topical Questionaire (Children, 12-17)           147
National Survey of Children's Health Topical Questionaire (Chilren, 6-11)             141
Medical Expenditure Panel Survey Insurance Component       

In [26]:
# How many surveys does each question appear in?
questions_per_survey = df_long.groupby('question_id').size()

print("\nDistribution of questions across surveys:")
print(questions_per_survey.value_counts().sort_index())
print(f"\nQuestions appearing in only 1 survey: {(questions_per_survey == 1).sum():,}")
print(f"Questions appearing in 2+ surveys: {(questions_per_survey >= 2).sum():,}")
print(f"Questions appearing in 5+ surveys: {(questions_per_survey >= 5).sum():,}")


Distribution of questions across surveys:
1    5954
2     266
3      67
4       5
5       1
6       1
7       2
Name: count, dtype: int64

Questions appearing in only 1 survey: 5,954
Questions appearing in 2+ surveys: 342
Questions appearing in 5+ surveys: 4


## Reorder Columns and Save

In [27]:
# Reorder columns for clarity
df_output = df_long[[
    'question_id',
    'Question',
    'survey',
    'is_multilevel',
    'question_length'
]].copy()

# Rename for consistency
df_output = df_output.rename(columns={'Question': 'question_text'})

print("Final columns:")
print(df_output.columns.tolist())
print(f"\nFinal shape: {df_output.shape}")
print("\nSample rows:")
print(df_output.head(20))

Final columns:
['question_id', 'question_text', 'survey', 'is_multilevel', 'question_length']

Final shape: (6732, 5)

Sample rows:
    question_id  \
0             1   
1             2   
2             3   
3             4   
4             5   
5             6   
6             7   
7             8   
8             9   
9            10   
10           11   
11           12   
12           13   
13           14   
14           15   
15           16   
16           17   
17           18   
18           19   
19           20   

                                                                                          question_text  \
0              At any time during this school year did you attend a public or private school in person?   
1   At any time during this school year did you receive any online schooling or virtual learning pro...   
2   At any time during this school year did you receive homeschooling instead of being enrolled in a...   
3     At any time during this school yea

In [28]:
# Save cleaned long-format data
output_path = '../data/processed/survey_questions_cleaned.csv'
df_output.to_csv(output_path, index=False)

print(f"\nCleaned data saved to: {output_path}")


Cleaned data saved to: ../data/processed/survey_questions_cleaned.csv


## Summary Statistics

In [29]:
print("=== DATA CLEANING SUMMARY ===")
print(f"\nInput: Wide format with {len(df)} questions across {len(survey_cols)} surveys")
print(f"Output: Long format with {len(df_output):,} question-survey pairs")
print(f"\nUnique questions: {df_output['question_id'].nunique():,}")
print(f"Unique surveys: {df_output['survey'].nunique()}")
print(f"\nData quality flags:")
print(f"  Multi-level question pairs: {num_multilevel:,} ({pct_multilevel:.1f}%)")
print(f"  Unique very short questions (<20 chars): {len(very_short):,}")
print(f"  Unique very long questions (>500 chars): {len(very_long):,}")
print(f"\nQuestions by survey count:")
print(f"  Appearing in 1 survey: {(questions_per_survey == 1).sum():,}")
print(f"  Appearing in 2-4 surveys: {((questions_per_survey >= 2) & (questions_per_survey < 5)).sum():,}")
print(f"  Appearing in 5+ surveys: {(questions_per_survey >= 5).sum():,}")
print(f"\nOutput file: {output_path}")

=== DATA CLEANING SUMMARY ===

Input: Wide format with 6733 questions across 48 surveys
Output: Long format with 6,732 question-survey pairs

Unique questions: 6,296
Unique surveys: 45

Data quality flags:
  Multi-level question pairs: 123 (1.8%)
  Unique very short questions (<20 chars): 22
  Unique very long questions (>500 chars): 116

Questions by survey count:
  Appearing in 1 survey: 5,954
  Appearing in 2-4 surveys: 338
  Appearing in 5+ surveys: 4

Output file: ../data/processed/survey_questions_cleaned.csv


## Notes for Next Steps

**Data format:** Long format with one row per question-survey pair, ready for embedding generation.

**Question IDs:** Each unique question has a consistent ID across all surveys where it appears.

**Multi-level questions:** Flagged but not split. Decision on handling will be made after clustering analysis.

**Next notebook:** Generate RoBERTa embeddings for unique questions (embed once, reuse for all surveys).